In [1]:
%%time
!pip install -qqq /kaggle/input/mmdetv3-env/archive/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmengine-0.7.4-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/weighted-boxes-fusion/ensemble_boxes-1.0.9-py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetection-3-1-evn/src/mmdet-3.1.0-py3-none-any.whl

CPU times: user 1.23 s, sys: 218 ms, total: 1.44 s
Wall time: 1min 31s


In [2]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc

from glob import glob
import matplotlib.pyplot as plt

import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import Text, Dict, Tuple
import zlib

import mmdet, mmcv, mmengine
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
#from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer


print(mmdet.__version__)
print(mmcv.__version__)
print(mmengine.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


3.1.0
2.0.0
0.7.4


In [3]:
%%writefile my_cmrcnn.py
load_from = '/kaggle/input/hubmap-mmdet-fold-1/best_coco_segm_mAP_epoch_16.pth'

model = dict(
    type='MaskRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_mask=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=-1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
#         init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet152')
    ),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.0, 0.0],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=1,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100,
            mask_thr_binary=0.5)))

dataset_type = 'CocoDataset'
data_root = 'data/coco/'
backend_args = None

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True, poly2mask=True),
    dict(type='Resize', scale=(1024,1024), keep_ratio=True),
    dict(type='RandomFlip', direction=['horizontal', 'vertical'], prob=0.6),
#     dict(type='Equalize',prob=0.5),
    dict(type='Sharpness'),
    dict(type='Contrast'),
    dict(type='Brightness'),
    dict(type='CutOut',n_holes=(1,5),cutout_shape=(10,10)),
    dict(type='PackDetInputs'),
 ]

test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
#     dict(type='LoadAnnotations', with_bbox=True, with_mask=True, poly2mask=True),
    dict(type='Resize', scale=(1024,1024), keep_ratio=True),
    dict(type='PackDetInputs',meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape','scale_factor'))
]        

train_dataloader = dict(
    batch_size=2,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type='CocoDataset',
        data_root='/kaggle/input/hubmap-hacking-the-human-vasculature',
        ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_train_class_1_folds5_fold1.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=train_pipeline,
        backend_args=None,
        metainfo=dict(classes='blood_vessel')))
val_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        data_root='/kaggle/input/hubmap-hacking-the-human-vasculature',
        ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
        data_prefix=dict(img='train/'),
        test_mode=True,
        pipeline=test_pipeline,
        backend_args=None,
        metainfo=dict(classes='blood_vessel')))
test_dataloader = dict(
    batch_size=2,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        data_root='/kaggle/input/hubmap-kidney-contest-dataset',
        ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
        data_prefix=dict(img='train/'),
        test_mode=True,
        pipeline=test_pipeline,
        backend_args=None,
        metainfo=dict(classes='blood_vessel')))
val_evaluator = dict(
    type='CocoMetric',
    ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
    metric=['segm'],
    format_only=False,
    backend_args=None,
#     iou_thrs=[0.6]
)
test_evaluator = dict(
    type='CocoMetric',
    ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
    metric=['segm'],
    format_only=False,
    backend_args=None,
#     iou_thrs=[0.6]
)

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=30, val_interval=1)
val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')


param_scheduler = [
    dict(
        type='LinearLR', start_factor=0.01, by_epoch=False, begin=0, end=300),
    dict(
        type='CosineAnnealingLR',
        begin=300,
        T_max = 1200,
        eta_min = 0.001*0.0025,
        by_epoch=False,
        verbose=False,
    ),  
#     dict(
#         type='ReduceOnPlateauLR',
#         monitor = 'coco/segm_mAP',
#         rule = 'greater',
#         factor = 0.1,
#         patience = 10,
#         threshold = 0.001,
#         begin=5,
#         min_value = 0.001*0.001,
#         by_epoch=True,
#         verbose=False,
#     )
]
optim_wrapper = dict(
    type='OptimWrapper',
#     optimizer=dict(
#         type='AdamW',
#         lr=0.0005,
#         weight_decay=0.0,
#     )
    optimizer=dict(type='SGD', lr=0.005,nesterov=True, momentum=0.90, weight_decay=0.005)
)

default_scope = 'mmdet'
default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    logger=dict(type='LoggerHook', interval=100),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook',interval=1,save_best='coco/segm_mAP'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'))

custom_hooks = [
    dict(
        type='EarlyStoppingHook',
        monitor='coco/segm_mAP',
        rule='greater',
        min_delta=0.0001,
        strict=False,
        check_finite=True,
        patience=20,
        stopping_threshold=None)
]

env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'))
vis_backends = [dict(type='LocalVisBackend')]
visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[dict(type='LocalVisBackend')],
    name='visualizer')
log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
log_level = 'INFO'
work_dir = '/kaggle/working/'

Writing my_cmrcnn.py


In [4]:
config = '/kaggle/working/my_cmrcnn.py'
cfg = Config.fromfile(config)
runner = Runner.from_cfg(cfg)

07/31 21:57:25 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 240408780
    GPU 0: Tesla P100-PCIE-16GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04.1) 11.3.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 11.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gen

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

07/31 21:57:45 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
07/31 21:57:45 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

In [5]:
runner.train()

07/31 21:57:48 - mmengine - WARNING - blood_vessel is not a meta file, simply parsed as meta information
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loads checkpoint by local backend from path: /kaggle/input/hubmap-mmdet-fold-1/best_coco_segm_mAP_epoch_16.pth
07/31 21:57:54 - mmengine - INFO - Load checkpoint from /kaggle/input/hubmap-mmdet-fold-1/best_coco_segm_mAP_epoch_16.pth
07/31 21:57:54 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/31 21:57:54 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
07/31 21:57:54 - mmengine - INFO - Checkpoints will be saved to /kaggle/working.
07/31 21:59:09 - mmengine 

MaskRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)

In [6]:
# %%writefile my_cmrcnn.py
# load_from = '/kaggle/input/hubmap-mmdet-fold-1/best_coco_segm_mAP_epoch_16.pth'
# model = dict(
#     type='MaskRCNN',
#     data_preprocessor=dict(
#         type='DetDataPreprocessor',
#         mean=[123.675, 116.28, 103.53],
#         std=[58.395, 57.12, 57.375],
#         bgr_to_rgb=True,
#         pad_mask=True,
#         pad_size_divisor=32),
#     backbone=dict(
#         type='ResNet',
#         depth=101,
#         num_stages=4,
#         out_indices=(0, 1, 2, 3),
#         frozen_stages=1,
#         norm_cfg=dict(type='BN', requires_grad=True),
#         norm_eval=False,
#         style='pytorch',
# #         init_cfg=dict(type='Pretrained',
# #                       checkpoint='torchvision://resnet101')
#     ),
#     neck=dict(
#         type='FPN',
#         in_channels=[256, 512, 1024, 2048],
#         out_channels=256,
#         num_outs=5),
#     rpn_head=dict(
#         type='RPNHead',
#         in_channels=256,
#         feat_channels=256,
#         anchor_generator=dict(
#             type='AnchorGenerator',
#             scales=[8],
#             ratios=[0.5, 1.0, 2.0],
#             strides=[4, 8, 16, 32, 64]),
#         bbox_coder=dict(
#             type='DeltaXYWHBBoxCoder',
#             target_means=[0.0, 0.0, 0.0, 0.0],
#             target_stds=[1.0, 1.0, 1.0, 1.0]),
#         loss_cls=dict(
#             type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
#         loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
#     roi_head=dict(
#         type='StandardRoIHead',
#         bbox_roi_extractor=dict(
#             type='SingleRoIExtractor',
#             roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
#             out_channels=256,
#             featmap_strides=[4, 8, 16, 32]),
#         bbox_head=dict(
#             type='Shared2FCBBoxHead',
#             in_channels=256,
#             fc_out_channels=1024,
#             roi_feat_size=7,
#             num_classes=1,
#             bbox_coder=dict(
#                 type='DeltaXYWHBBoxCoder',
#                 target_means=[0.0, 0.0, 0.0, 0.0],
#                 target_stds=[0.1, 0.1, 0.2, 0.2]),
#             reg_class_agnostic=False,
#             loss_cls=dict(
#                 type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
#             loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
#         mask_roi_extractor=dict(
#             type='SingleRoIExtractor',
#             roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
#             out_channels=256,
#             featmap_strides=[4, 8, 16, 32]),
#         mask_head=dict(
#             type='FCNMaskHead',
#             num_convs=4,
#             in_channels=256,
#             conv_out_channels=256,
#             num_classes=1,
#             loss_mask=dict(
#                 type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
#     train_cfg=dict(
#         rpn=dict(
#             assigner=dict(
#                 type='MaxIoUAssigner',
#                 pos_iou_thr=0.7,
#                 neg_iou_thr=0.3,
#                 min_pos_iou=0.3,
#                 match_low_quality=True,
#                 ignore_iof_thr=-1),
#             sampler=dict(
#                 type='RandomSampler',
#                 num=256,
#                 pos_fraction=0.5,
#                 neg_pos_ub=-1,
#                 add_gt_as_proposals=False),
#             allowed_border=-1,
#             pos_weight=-1,
#             debug=False),
#         rpn_proposal=dict(
#             nms_across_levels=False,
#             nms_pre=2000,
#             nms_post=1000,
#             max_per_img=1000,
#             nms=dict(type='nms', iou_threshold=0.7),
#             min_bbox_size=0),
#         rcnn=dict(
#             assigner=dict(
#                 type='MaxIoUAssigner',
#                 pos_iou_thr=0.6,
#                 neg_iou_thr=0.6,
#                 min_pos_iou=0.6,
#                 match_low_quality=False,
#                 ignore_iof_thr=-1),
#             sampler=dict(
#                 type='RandomSampler',
#                 num=512,
#                 pos_fraction=0.25,
#                 neg_pos_ub=-1,
#                 add_gt_as_proposals=True),
#             mask_size=28,
#             pos_weight=-1,
#             debug=False)),
#     test_cfg=dict(
#         rpn=dict(
#             nms_across_levels=False,
#             nms_pre=1000,
#             nms_post=1000,
#             max_per_img=1000,
#             nms=dict(type='nms', iou_threshold=0.7),
#             min_bbox_size=0),
#         rcnn=dict(
#             score_thr=0.05,
#             nms=dict(type='nms', iou_thr=0.6),
#             max_per_img=100,
#             mask_thr_binary=0.5)))
# dataset_type = 'CocoDataset'
# data_root = ''
# backend_args = None
# metainfo = dict(classes=('blood_vessel', ), palette=[(255, 0, 0)])
# train_pipeline = [
#     dict(type='LoadImageFromFile', backend_args=None),
#     dict(
#         type='LoadAnnotations', with_bbox=True, with_mask=True,
#         poly2mask=True),
#     dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
#     dict(type='RandomFlip', direction=['horizontal', 'vertical'], prob=0.6),
#     dict(type='PackDetInputs')
# ]
# test_pipeline = [
#     dict(type='LoadImageFromFile', backend_args=None),
#     dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
#     dict(
#         type='PackDetInputs',
#         meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
#                    'scale_factor'))
# ]
# train_dataloader = dict(
#     batch_size=2,
#     num_workers=2,
#     persistent_workers=True,
#     sampler=dict(type='DefaultSampler', shuffle=True),
#     batch_sampler=dict(type='AspectRatioBatchSampler'),
#     dataset=dict(
#         type='CocoDataset',
#         metainfo=dict(classes=('blood_vessel', ), palette=[(255, 0, 0)]),
#         data_root='',
#         ann_file='/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_train_class_1_folds5_fold1.json',
#         data_prefix=dict(
#             img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
#         filter_cfg=dict(filter_empty_gt=True, min_size=32),
#         pipeline=[
#             dict(type='LoadImageFromFile', backend_args=None),
#             dict(
#                 type='LoadAnnotations',
#                 with_bbox=True,
#                 with_mask=True,
#                 poly2mask=True),
#             dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
#             dict(
#                 type='RandomFlip',
#                 direction=['horizontal', 'vertical'],
#                 prob=0.5),
#             dict(type='PackDetInputs')
#         ],
#         backend_args=None))
# val_dataloader = dict(
#     batch_size=4,
#     num_workers=2,
#     persistent_workers=True,
#     drop_last=False,
#     sampler=dict(type='DefaultSampler', shuffle=False),
#     dataset=dict(
#         type='CocoDataset',
#         metainfo=dict(classes=('blood_vessel', ), palette=[(255, 0, 0)]),
#         data_root='',
#         ann_file=
#         '/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
#         data_prefix=dict(
#             img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
#         test_mode=True,
#         pipeline=[
#             dict(type='LoadImageFromFile', backend_args=None),
#             dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
#             dict(
#                 type='PackDetInputs',
#                 meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
#                            'scale_factor'))
#         ],
#         backend_args=None))
# test_dataloader = dict(
#     batch_size=4,
#     num_workers=2,
#     persistent_workers=True,
#     drop_last=False,
#     sampler=dict(type='DefaultSampler', shuffle=False),
#     dataset=dict(
#         type='CocoDataset',
#         metainfo=dict(classes=('blood_vessel', ), palette=[(255, 0, 0)]),
#         data_root='',
#         ann_file=
#         '/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
#         data_prefix=dict(
#             img='/kaggle/input/hubmap-hacking-the-human-vasculature/train/'),
#         test_mode=True,
#         pipeline=[
#             dict(type='LoadImageFromFile', backend_args=None),
#             dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
#             dict(
#                 type='PackDetInputs',
#                 meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
#                            'scale_factor'))
#         ],
#         backend_args=None))
# val_evaluator = dict(
#     type='CocoMetric',
#     ann_file=
#     '/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
#     metric=['segm'],
#     format_only=False,
#     backend_args=None,
# #     iou_thrs=[0.6]
# )
# test_evaluator = dict(
#     type='CocoMetric',
#     ann_file=
#     '/kaggle/input/pure-5-fold-dataset/coco_annotations_PURE_valid_class_1_folds5_fold1.json',
#     metric=['segm'],
#     format_only=False,
#     backend_args=None,
# #     iou_thrs=[0.6]
# )
# optim_wrapper = dict(
#     type='OptimWrapper',
#     optimizer=dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.005),
#     clip_grad=None)
# param_scheduler = [
#     dict(
#         type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=320),
#     dict(
#         type='MultiStepLR',
#         by_epoch=True,
#         begin=0,
#         end=25,
#         milestones=[13, 17],
#         gamma=0.1)
# ]
# default_hooks = dict(
#     timer=dict(type='IterTimerHook'),
#     logger=dict(type='LoggerHook', interval=50),
#     param_scheduler=dict(type='ParamSchedulerHook'),
#     checkpoint=dict(
#         type='CheckpointHook', interval=1, save_best='coco/segm_mAP'),
#     sampler_seed=dict(type='DistSamplerSeedHook'),
#     visualization=dict(type='DetVisualizationHook'))
# custom_hooks = [
#     dict(
#         type='EarlyStoppingHook',
#         monitor='coco/segm_mAP',
#         rule='greater',
#         min_delta=0.005,
#         strict=False,
#         check_finite=True,
#         patience=15,
#         stopping_threshold=None)
# ]
# default_scope = 'mmdet'
# env_cfg = dict(
#     cudnn_benchmark=False,
#     mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
#     dist_cfg=dict(backend='nccl'))
# vis_backends = [dict(type='LocalVisBackend')]
# visualizer = dict(
#     type='DetLocalVisualizer',
#     vis_backends=[dict(type='LocalVisBackend')],
#     name='visualizer')
# log_config = dict(hooks=[
#     dict(type='TextLoggerHook'),
#     dict(
#         type='MMDetWandbHook',
#         init_kwargs=dict(project='mmdetection'),
#         interval=10,
#         log_checkpoint=True,
#         log_checkpoint_metadata=True,
#         num_eval_images=100,
#         bbox_score_thr=0.3)
# ])
# log_processor = dict(type='LogProcessor', window_size=50, by_epoch=True)
# log_level = 'INFO'
# load_from = None
# resume = False
# train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=25, val_interval=1)
# val_cfg = dict(type='ValLoop')
# test_cfg = dict(type='TestLoop')
# work_dir = '/kaggle/working/work_dir'

In [7]:
# config = '/kaggle/working/my_cmrcnn.py'
# cfg = Config.fromfile(config)
# runner = Runner.from_cfg(cfg)

In [8]:
# runner.train()